In [1]:
import numpy as np
import pandas as pd

import json

import gspread
from oauth2client.service_account import ServiceAccountCredentials

# fxns
def getDexNum(mon):
    num = int(db[db['Pokemon'] == mon].index.values[0])
    return(num)

def makeRaidInfo(dexNum):
    
    row = db[db.index.values == dexNum]
        
    mon = row['Pokemon'].values[0]
    cpRange = "{}-{}".format(str(row['CP Lv20 (10/10/10)'].values[0]), 
                             str(row['CP Lv20 (15/15/15)'].values[0]))
    boostRange = "{}-{}".format(str(row['CP Lv25 (10/10/10)'].values[0]),
                                str(row['CP Lv25 (15/15/15)'].values[0]))

    return(mon, boostRange, cpRange)

def makeTierInfo(bossList):
    
    dexList = []
    for mon in bossList:
        num = getDexNum(mon)
        dexList.append(num)
        
    dexList.sort()
        
    boostList = []
    cpList = []
    
    for dexN in dexList:
        mon, boost, cp = makeRaidInfo(dexN)
        boostList.append(boost)
        cpList.append(cp)

    return(dexList, boostList, cpList)

def setJsonParams(bossDict, hatchtime, raidtime):
    
    t1d, t1b, t1c = makeTierInfo(bossDict['1'])
    t2d, t2b, t2c = makeTierInfo(bossDict['2'])
    t3d, t3b, t3c = makeTierInfo(bossDict['3'])
    t4d, t4b, t4c = makeTierInfo(bossDict['4'])
    t5d, t5b, t5c = makeTierInfo(bossDict['5'])
    EXd, EXb, EXc = makeTierInfo(bossDict['EX'])
    
    raid_info = {
        "raid_eggs": {
            "1": {
                "egg":"normal",
                "egg_img": "1.png",
                "pokemon": t1d,
                "boost_range": t1b,
                "cp_range": t1c,
                "hatchtime": hatchtime,
                "raidtime": raidtime
            },
            "2": {
                "egg":"normal",
                "egg_img": "2.png",
                "pokemon": t2d,
                "boost_range": t2b,
                "cp_range": t2c,
                "hatchtime": hatchtime,
                "raidtime": raidtime
            },
            "3": {
                "egg":"rare",
                "egg_img": "3.png",
                "pokemon": t3d,
                "boost_range": t3b,
                "cp_range": t3c,
                "hatchtime": hatchtime,
                "raidtime": raidtime
            },
            "4": {
                "egg":"rare",
                "egg_img": "4.png",
                "pokemon": t4d,
                "boost_range": t4b,
                "cp_range": t4c,
                "hatchtime": hatchtime,
                "raidtime": raidtime
            },
            "5": {
                "egg":"legendary",
                "egg_img": "5.png",
                "pokemon": t5d,
                "boost_range": t5b,
                "cp_range": t5c,
                "hatchtime": hatchtime,
                "raidtime": raidtime
            },
            "EX": {
                "egg":"EX",
                "egg_img": "EX.png",
                "pokemon": EXd,
                "boost_range": EXb,
                "cp_range": EXc,
                "hatchtime": hatchtime,
                "raidtime": raidtime
            }
        }
    }
    
    return(raid_info)

# sets scope, credentials, client - don't mess with this
scope = ['https://spreadsheets.google.com/feeds',
        'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('squirtudo-496dea56c402.json', scope)
client = gspread.authorize(creds)

# sheet name: squirtudo client has perms for all sheets in our shared folder
sheet = client.open('raid-boss-database').sheet1

# data is a list of dicts, each of which is a row
data = sheet.get_all_records()

# convert to dataframe for ez manipulation
db = pd.DataFrame(data).set_index('Num')

# Change these params:

In [2]:
bossDict = {'1': ['Mareep','Magnemite','Shinx','Buizel'],
           '2': ['Lanturn', 'Manectric','Electabuzz','Mawile'],
           '3': ['Skarmory','Scyther','Alolan Raichu','Jolteon'],
           '4': ['Alolan Exeggutor','Aggron','Alolan Marowak','Absol'],
           '5': ['Giratina'],
           'EX': ['Deoxys']}

raid_info = setJsonParams(bossDict, hatchtime=60, raidtime=45)

with open('../../data/raid_info.json', 'w') as outfile:  
    json.dump(raid_info, outfile)